In [1]:
import osmnx as ox;
import folium;
import pandas as pd;
from folium.plugins import FloatImage
import base64
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.point import Point

In [2]:
place = 'Bengaluru'

In [ ]:
#Tags acquired from 'https://wiki.openstreetmap.org/wiki/Buildings'

community_centre = {'amenity':'community_centre'}
library = {'amenity':'library'}
community_building = ox.geometries_from_place(place, community_centre)
library_building = ox.geometries_from_place(place, library)

In [ ]:
community_building.head()

In [ ]:
print(len(community_building.index)) #All features in community_building = 166

community_building_points = community_building[community_building.geom_type == 'Point'][:100] #Only Point features in community_building = 85
print(len(community_building_points.index))

community_building_polygon = community_building[community_building.geom_type == 'Polygon'][:100] #Only Polygon features in community_building = 81
print(len(community_building_polygon.index))


In [ ]:
community_building_points.head()

In [8]:
locator = Nominatim(user_agent="google")

In [ ]:
community_building_points["geocode"] = community_building_points['geometry'].y.map(str) + ',' + community_building_points['geometry'].x.map(str)
for ind in community_building_points.index:
    location = locator.reverse(community_building_points["geocode"][ind])
    community_building_points['geocode'][ind] = location.address

In [ ]:
community_building_polygon["centroid"] = community_building_polygon.to_crs(epsg=4326).centroid.to_crs(community_building_polygon.crs)

In [ ]:
community_building_polygon["geocode"] = community_building_polygon['centroid'].y.map(str) + ',' + community_building_polygon['centroid'].x.map(str)
for ind in community_building_polygon.index:
    location = locator.reverse(community_building_polygon["geocode"][ind])
    community_building_polygon['geocode'][ind] = location.address
community_building_polygon = community_building_polygon.drop('centroid', axis=1)

In [ ]:
library_building.head()

In [ ]:
print(len(library_building.index)) #All features in community_building = 76

library_building_points = library_building[library_building.geom_type == 'Point'][:100] #Only Point features in community_building = 51
print(len(library_building_points.index))

library_building_polygon = library_building[library_building.geom_type == 'Polygon'][:100] #Only Polygon features in community_building = 25
print(len(library_building_polygon.index))

In [ ]:
library_building_points["geocode"] = library_building_points['geometry'].y.map(str) + ',' + library_building_points['geometry'].x.map(str)
for ind in library_building_points.index:
    location = locator.reverse(library_building_points["geocode"][ind])
    library_building_points['geocode'][ind] = location.address

In [ ]:
library_building_polygon["centroid"] = library_building_polygon.to_crs(epsg=4326).centroid.to_crs(library_building_polygon.crs)

In [ ]:
library_building_polygon["geocode"] = library_building_polygon['centroid'].y.map(str) + ',' + library_building_polygon['centroid'].x.map(str)
for ind in library_building_polygon.index:
    location = locator.reverse(library_building_polygon["geocode"][ind])
    library_building_polygon['geocode'][ind] = location.address
library_building_polygon = library_building_polygon.drop('centroid', axis=1)

In [ ]:
icon1 = folium.Icon(color='green',prefix='glyphicon',icon='home')
icon2 = folium.Icon(color='#FF7377 ',prefix='glyphicon',icon='book')

In [ ]:

m = folium.Map([12.9716, 77.5946], zoom_start=15, tiles='cartodbpositron', width=1800,height=1000)

folium.GeoJson(community_building_points,
               style_function=lambda x, mean_rsrp=community_building: {
                     'fillColor': 'yellow',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon1),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(library_building_points,
               style_function=lambda x, mean_rsrp=library_building: {
                     'fillColor': 'blue',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon2),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(community_building_polygon,
               style_function=lambda x, mean_rsrp=community_building: {
                     'fillColor': 'yellow',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)
folium.GeoJson(library_building_polygon,
               style_function=lambda x, mean_rsrp=library_building: {
                     'fillColor': 'blue',
                     'color': 'black',
                     'weight': 1,
                     'fillOpacity': 1
                  },
                marker =folium.Marker([45.5, -122.6], icon=icon2),
                tooltip=folium.GeoJsonTooltip(fields=['geocode'])
               ).add_to(m)

In [ ]:
image_file = 'Legend.png'

FloatImage(image_file, left=80).add_to(m)

In [ ]:
m

In [39]:
m.save('GeoBLR.html')